In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# setup the mesh meta to handle downloads and caching for cell bodies
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching for mitochondria
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

# setup the mesh meta to handle downloads and caching for nuclei
nuc_mesh_dir = 'data/nuclei_meshes/'
nuc_source = "precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-nuclei/seg"
nuc_mm = trimesh_io.MeshMeta(cv_path=nuc_source,
                         disk_cache_path=nuc_mesh_dir)


In [6]:
nuc = pd.read_csv("data/230106_soma_valence_v185.csv", index_col=[0])
nuc

,id,cell_type,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,nuc_id
0,17,e,[103860 44385 891],648518346349540057,415440,177540,35640,4809
1,453,e,[84054 50604 1424],648518346349538466,336216,202416,56960,6097
2,35,e,[103728 54333 304],648518346349538711,414912,217332,12160,6582
3,74,e,[51543 57568 155],648518346349538715,206172,230272,6200,7498
4,231,e,[77197 61773 428],648518346349537038,308788,247092,17120,8610
...,...,...,...,...,...,...,...,...
451,298,e,[116314 58234 933],648518346349537961,465256,232936,37320,7570
452,444,e,[70253 54531 1752],648518346349538336,281012,218124,70080,6877
453,69,e,[75256 56402 571],648518346349537509,301024,225608,22840,7525
454,116,e,[77180 55980 185],648518346349537641,308720,223920,7400,6557


In [7]:
zeroes = nuc[nuc['nuc_id'] == 0]
zeroes

,id,cell_type,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,nuc_id
86,196,e,[62316 63307 2085],648518346349539390,249264,253228,83400,0
172,315,e,[100322 38374 2074],648518346349493874,401288,153496,82960,0
196,346,i,[70079 48099 356],648518346349515985,280316,192396,14240,0
299,288,e,[52102 44781 1091],648518346349523667,208408,179124,43640,0
356,362,i,[99769 54014 2156],648518346349522740,399076,216056,86240,0
427,262,e,[110245 59258 92],648518346349536391,440980,237032,3680,0


In [8]:
nuc_removezeroes = nuc[nuc['nuc_id'] != 0]
nuc_removezeroes

,id,cell_type,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,nuc_id
0,17,e,[103860 44385 891],648518346349540057,415440,177540,35640,4809
1,453,e,[84054 50604 1424],648518346349538466,336216,202416,56960,6097
2,35,e,[103728 54333 304],648518346349538711,414912,217332,12160,6582
3,74,e,[51543 57568 155],648518346349538715,206172,230272,6200,7498
4,231,e,[77197 61773 428],648518346349537038,308788,247092,17120,8610
...,...,...,...,...,...,...,...,...
451,298,e,[116314 58234 933],648518346349537961,465256,232936,37320,7570
452,444,e,[70253 54531 1752],648518346349538336,281012,218124,70080,6877
453,69,e,[75256 56402 571],648518346349537509,301024,225608,22840,7525
454,116,e,[77180 55980 185],648518346349537641,308720,223920,7400,6557


In [9]:
nuc_removezeroes.cell_type.value_counts()

e            358
g             59
i             32
uncertain      1
Name: cell_type, dtype: int64

In [10]:
nuc_all_list = nuc_removezeroes.nuc_id.to_list()

In [11]:
len(nuc_all_list)

450

In [12]:
nuc_all_list.sort()

In [13]:
nuc_all_list

[123,
 375,
 642,
 1318,
 1889,
 1890,
 1892,
 1911,
 1914,
 2256,
 2730,
 3118,
 3181,
 3511,
 3531,
 3536,
 3539,
 3540,
 3541,
 3542,
 3543,
 3545,
 3546,
 3550,
 3551,
 3554,
 3564,
 3567,
 3575,
 3670,
 3673,
 3798,
 3854,
 4029,
 4306,
 4308,
 4309,
 4310,
 4312,
 4313,
 4316,
 4317,
 4330,
 4332,
 4333,
 4335,
 4336,
 4337,
 4338,
 4339,
 4340,
 4341,
 4342,
 4344,
 4349,
 4351,
 4369,
 4378,
 4381,
 4597,
 4624,
 4762,
 4763,
 4764,
 4766,
 4768,
 4769,
 4774,
 4775,
 4778,
 4782,
 4784,
 4786,
 4793,
 4794,
 4795,
 4796,
 4797,
 4799,
 4801,
 4802,
 4804,
 4805,
 4808,
 4809,
 4810,
 4811,
 4812,
 4813,
 4815,
 4820,
 4887,
 5080,
 5081,
 5084,
 5085,
 5086,
 5088,
 5092,
 5093,
 5116,
 5121,
 5123,
 5125,
 5126,
 5128,
 5130,
 5131,
 5132,
 5134,
 5135,
 5136,
 5137,
 5143,
 5145,
 5146,
 5156,
 5158,
 5160,
 5161,
 5162,
 5327,
 5441,
 5626,
 5629,
 5631,
 5637,
 5639,
 5640,
 5642,
 5643,
 5648,
 5649,
 5650,
 5651,
 5652,
 5660,
 5672,
 5680,
 5682,
 5684,
 5689,
 5692,
 5

In [14]:
# download nuclei meshes to local drive as specified above

for i in range(len(nuc_all_list)):
    nuc_seg_id = nuc_all_list[i]
    nuc_downloadmesh = nuc_mm.mesh(seg_id = nuc_seg_id, remove_duplicate_vertices=True)


In [15]:
# manual download for cells not in the soma table
# nuc_id not in soma table: 7986,7987

manual_nucid = 7986
nuc_downloadmesh = nuc_mm.mesh(seg_id = manual_nucid, remove_duplicate_vertices=True)


In [16]:
# iterate the nuc_mesh for each nuclei in list
# from https://python-forum.io/thread-23500.html

nuc_iterator = {}
for i in range(len(nuc_all_list)):
    nuc_seg_id = nuc_all_list[i] 
    nuc_mesh_file = os.path.join(nuc_mesh_dir, str(nuc_seg_id)+'.h5')
    nuc_iterator['nuc_mesh_' + str(i)] = nuc_mm.mesh(filename = nuc_mesh_file)
    
locals().update(nuc_iterator)

In [17]:
# opacity and color settings for nuclei meshes
cell_opac = 0.025 # 0.075
cell_color = (0.7, 0.7, 0.7) # (0.2, 0.4, 0.7) blue
nuc_opac = 0.40 # 0.35

# cell membrane mesh
#mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes
#actor_iterator = {}
#for i in range(len(mito_query_mitolist)):
#    random_color = list(np.random.random(size=3))
#    actor_iterator['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator['mito_mesh_'+str(i)], opacity=mito_opac, color=(random_color))
    
#locals().update(actor_iterator)

# nuclei meshes
nuc_actor_iterator = {}
for i in range(len(nuc_all_list)):
    random_color = list(np.random.random(size=3))
    nuc_actor_iterator['nuc_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(nuc_iterator['nuc_mesh_'+str(i)], opacity=nuc_opac, color=(random_color))
    
locals().update(nuc_actor_iterator)


# update dictionary for the render actors code below
#actor_iterator['mesh_actor'] = mesh_actor

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera((nuc_mesh_165.centroid), backoff=500) # this nuclei is near the center of the volume

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(nuc_actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001E38D286230) at 0x000001E3E600C4C8>

In [18]:
# color by cell type (e,i,g)

In [19]:
exc = nuc_removezeroes[nuc_removezeroes['cell_type'] == 'e']
exc_list = exc.nuc_id.to_list()
len(exc_list)

358

In [20]:
inh = nuc_removezeroes[nuc_removezeroes['cell_type'] == 'i']
inh_list = inh.nuc_id.to_list()
len(inh_list)

32

In [21]:
glia = nuc_removezeroes[nuc_removezeroes['cell_type'] == 'g']
glia_list = glia.nuc_id.to_list()
len(glia_list)

59

In [22]:
# iterate the nuc_mesh for each nuclei in list by cell type
# from https://python-forum.io/thread-23500.html

exc_iterator = {}
for i in range(len(exc_list)):
    nuc_seg_id = exc_list[i] 
    nuc_mesh_file = os.path.join(nuc_mesh_dir, str(nuc_seg_id)+'.h5')
    exc_iterator['exc_mesh_' + str(i)] = nuc_mm.mesh(filename = nuc_mesh_file)

locals().update(exc_iterator)

inh_iterator = {}
for i in range(len(inh_list)):
    nuc_seg_id = inh_list[i] 
    nuc_mesh_file = os.path.join(nuc_mesh_dir, str(nuc_seg_id)+'.h5')
    inh_iterator['inh_mesh_' + str(i)] = nuc_mm.mesh(filename = nuc_mesh_file)

locals().update(inh_iterator)

glia_iterator = {}
for i in range(len(glia_list)):
    nuc_seg_id = glia_list[i] 
    nuc_mesh_file = os.path.join(nuc_mesh_dir, str(nuc_seg_id)+'.h5')
    glia_iterator['glia_mesh_' + str(i)] = nuc_mm.mesh(filename = nuc_mesh_file)

locals().update(glia_iterator)


In [23]:
# opacity and color settings for nuclei meshes
exc_opac = 0.10 # 0.075
exc_color = (0.2, 0.4, 0.7) # (0.2, 0.4, 0.7) blue
inh_opac = 0.10 # 0.35
inh_color = (0.7, 0.4, 0.2)
glia_opac = 0.75
glia_color = (0.7, 0.7, 0.2)

# exc meshes
exc_actor_iterator = {}
for i in range(len(exc_list)):
    #random_color = list(np.random.random(size=3))
    exc_actor_iterator['exc_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(exc_iterator['exc_mesh_'+str(i)], opacity=exc_opac, color=exc_color)
    
locals().update(exc_actor_iterator)

# inh meshes
inh_actor_iterator = {}
for i in range(len(inh_list)):
    #random_color = list(np.random.random(size=3))
    inh_actor_iterator['inh_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(inh_iterator['inh_mesh_'+str(i)], opacity=inh_opac, color=inh_color)
    
locals().update(inh_actor_iterator)

# glia meshes
glia_actor_iterator = {}
for i in range(len(glia_list)):
    #random_color = list(np.random.random(size=3))
    glia_actor_iterator['glia_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(glia_iterator['glia_mesh_'+str(i)], opacity=glia_opac, color=glia_color)
    
locals().update(glia_actor_iterator)


# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator.update(exc_actor_iterator)
actor_iterator.update(inh_actor_iterator)
actor_iterator.update(glia_actor_iterator)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera((nuc_mesh_165.centroid), backoff=500) # this nuclei is near the center of the volume

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001E38D282FB0) at 0x000001E3E22363A8>

In [24]:
# add vascular ids
# load cell table

celltypes = pd.read_csv('data/220309_cell_type_classification.csv', index_col=[0])
celltypes

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
1,41760,60256,1702,648518346349538237,pyramidal,pyramidal cell
2,98128,39856,218,648518346349536816,pyramidal,pyramidal cell
3,51248,50608,86,648518346349539797,pyramidal,pyramidal cell
5,115392,47136,617,648518346349536744,pyramidal,pyramidal cell
6,104928,72496,1448,648518346349536851,pyramidal,pyramidal cell
...,...,...,...,...,...,...
68,107312,43552,2160,648518346349375478,vasculature,endothelial
69,104752,40768,878,648518346349061101,vasculature,endothelial
70,104464,40784,1867,648518346349375478,vasculature,endothelial
71,97377,38012,1145,648518346349375478,vasculature,endothelial


In [25]:
vasc = celltypes[celltypes['cell_type'] == 'vasculature']
vasc

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
0,113904,67296,24,648518346342804483,vasculature,endothelial
1,44512,71312,94,648518346341422370,vasculature,endothelial
2,76944,58784,1987,648518346349526102,vasculature,pericyte
3,104256,40800,1968,648518346349375478,vasculature,endothelial
4,45168,69328,105,648518346343497543,vasculature,endothelial
...,...,...,...,...,...,...
68,107312,43552,2160,648518346349375478,vasculature,endothelial
69,104752,40768,878,648518346349061101,vasculature,endothelial
70,104464,40784,1867,648518346349375478,vasculature,endothelial
71,97377,38012,1145,648518346349375478,vasculature,endothelial


In [26]:
vasc.cell_segid.unique()

array([648518346342804483, 648518346341422370, 648518346349526102,
       648518346349375478, 648518346343497543, 648518346348189552,
       648518346344072053, 648518346348121670, 648518346344328018,
       648518346344456391, 648518346344505300, 648518346341763956,
       648518346345118988, 648518346342104735, 648518346343459827,
       648518346342796989, 648518346349061101], dtype=int64)

In [27]:
len(vasc.cell_segid.unique())

17

In [28]:
endo = celltypes[celltypes['cell_subtype'] == 'endothelial']
endo

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
0,113904,67296,24,648518346342804483,vasculature,endothelial
1,44512,71312,94,648518346341422370,vasculature,endothelial
3,104256,40800,1968,648518346349375478,vasculature,endothelial
4,45168,69328,105,648518346343497543,vasculature,endothelial
5,107808,45904,2158,648518346348189552,vasculature,endothelial
...,...,...,...,...,...,...
68,107312,43552,2160,648518346349375478,vasculature,endothelial
69,104752,40768,878,648518346349061101,vasculature,endothelial
70,104464,40784,1867,648518346349375478,vasculature,endothelial
71,97377,38012,1145,648518346349375478,vasculature,endothelial


In [29]:
endo.cell_segid.unique()

array([648518346342804483, 648518346341422370, 648518346349375478,
       648518346343497543, 648518346348189552, 648518346344072053,
       648518346349526102, 648518346348121670, 648518346344328018,
       648518346344456391, 648518346344505300, 648518346341763956,
       648518346345118988, 648518346342104735, 648518346343459827,
       648518346342796989, 648518346349061101], dtype=int64)

In [30]:
len(endo.cell_segid.unique())

17

In [31]:
endo_list = endo.cell_segid.unique().tolist()

In [32]:
type(endo_list)

list

In [33]:
# download the cell meshes for the endothelial cells

for i in range(len(endo_list)):
    endo_id = endo_list[i]
    endo_seg_id = endo_id
    endo_downloadmesh = mm.mesh(seg_id = endo_seg_id, remove_duplicate_vertices=True)

In [34]:
# iterate the cell meshes for endothelial cells
# from https://python-forum.io/thread-23500.html

endo_iterator = {}
for i in range(len(endo_list)):
    endo_seg_id = endo_list[i] 
    endo_mesh_file = os.path.join(mesh_dir, str(endo_seg_id)+'.h5')
    endo_iterator['endo_mesh_' + str(i)] = mm.mesh(filename = endo_mesh_file)
    
locals().update(endo_iterator)

In [35]:
# opacity and color settings for nuclei meshes
exc_opac = 0.10 # 0.075
exc_color = (0.2, 0.4, 0.7) # (0.2, 0.4, 0.7) blue
inh_opac = 0.10 # 0.35
inh_color = (0.7, 0.4, 0.2)
glia_opac = 0.75
glia_color = (0.7, 0.7, 0.2)
vasc_opac = 0.15
vasc_color = (1,1,1)

# exc meshes
exc_actor_iterator = {}
for i in range(len(exc_list)):
    #random_color = list(np.random.random(size=3))
    exc_actor_iterator['exc_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(exc_iterator['exc_mesh_'+str(i)], opacity=exc_opac, color=exc_color)
    
locals().update(exc_actor_iterator)

# inh meshes
inh_actor_iterator = {}
for i in range(len(inh_list)):
    #random_color = list(np.random.random(size=3))
    inh_actor_iterator['inh_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(inh_iterator['inh_mesh_'+str(i)], opacity=inh_opac, color=inh_color)
    
locals().update(inh_actor_iterator)

# glia meshes
glia_actor_iterator = {}
for i in range(len(glia_list)):
    #random_color = list(np.random.random(size=3))
    glia_actor_iterator['glia_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(glia_iterator['glia_mesh_'+str(i)], opacity=glia_opac, color=glia_color)
    
locals().update(glia_actor_iterator)

# endothelial meshes
endo_actor_iterator = {}
for i in range(len(endo_list)):
    #random_color = list(np.random.random(size=3))
    endo_actor_iterator['endo_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(endo_iterator['endo_mesh_'+str(i)], opacity=vasc_opac, color=vasc_color)
    
locals().update(endo_actor_iterator)

# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator.update(exc_actor_iterator)
actor_iterator.update(inh_actor_iterator)
actor_iterator.update(glia_actor_iterator)
actor_iterator.update(endo_actor_iterator)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera((nuc_mesh_165.centroid), backoff=500) # this nuclei is near the center of the volume

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000001E38D280380) at 0x000001E43B73E828>